Identificação dos itens existentes no catálogo (arquivo items_titles.csv) mais similares aos novos itens (arquivo items_titles_test.csv)

Abordagem utilizada: 
Clusterizar cada item no catálogo. O cálculo da similaridade é realizado apenas com os itens do catálogo que pertencem ao mesmo cluster 

#### Funções de preparação dos dados  

In [3]:
def remover_palavras(data , lista_remocao , coluna_texto , nova_coluna_texto ): #Remover stopwords
   
    token_espaco = tokenize.WordPunctTokenizer()

    for i, row in data.iterrows():
        frase = list()
        for palavra in token_espaco.tokenize(row[coluna_texto]):
            if palavra not in lista_remocao:
                frase.append(palavra)
        data.at[i , nova_coluna_texto ] = ' '.join(frase)
        
def ajustes_diversos_dominio(data , coluna_texto , nova_coluna_texto , tamanho_minimo ): #Remover stopwords
    import re
    token_espaco = tokenize.WordPunctTokenizer()

    for i, row in data.iterrows():
        frase = list()
        for palavra in token_espaco.tokenize(row[coluna_texto]):
            palavra = re.sub(r"[!\"'#$%&()*+,-\.\/:;<=>?@[\\\]^_`{|}~]", ' ', palavra)
            if(palavra == 'tv'):
                frase.append('televisao')
            else:
                if len(palavra) > tamanho_minimo:
                    frase.append(palavra)
        data.at[i , nova_coluna_texto ] = ' '.join(frase)
                
        
# =============================================================================
# Aplica as tranformações necessárias para normalizar o texto da coluna text_col do DataFrame df
# Retorna um novo DF com uma coluna adicional com o nome de text_col + '_transf'
# =============================================================================
def preparar_dados(df , text_col):
    df_transf = pd.DataFrame()
    df_transf[text_col] = df[text_col]
    
    df_transf['trat1_lowercase'] = [unidecode.unidecode(texto.lower()) for texto in df_transf[text_col]]
    ajustes_diversos_dominio(df_transf , "trat1_lowercase" , "trat2_diversos" , tamanho_minimo=2 )
    
    palavras_irrelevantes = nltk.corpus.stopwords.words("portuguese")
    remover_palavras(df_transf , palavras_irrelevantes ,  "trat2_diversos" , "trat3_sem_sw" )

    #removendo linha inválida no dataset
    df_transf = df_transf.loc[df_transf['trat3_sem_sw'] != '']

    lemm = nltk.stem.RSLPStemmer()
    df_transf[text_col + '_transf'] = [lemm.stem(palavra) for palavra in df_transf['trat3_sem_sw']]
    
    return df_transf.drop( ['trat1_lowercase' , 'trat2_diversos' , 'trat3_sem_sw' ] , axis = 1)
    

Leitura dos itens do catálogo e preparação dos dados

In [4]:
import pandas as pd
import nltk
from nltk import tokenize
import unidecode
from sklearn.feature_extraction.text import TfidfVectorizer
from time import time

avaliacao_performance = pd.DataFrame(columns=['Descricao' , 'Tempo'] , index=[0])

nltk.download('stopwords')

df_base = pd.read_csv("items_titles.csv")
df_base = preparar_dados(df_base , 'ITE_ITEM_TITLE')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ulf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df_base.head()

,ITE_ITEM_TITLE,ITE_ITEM_TITLE_transf
0,Tênis Ascension Posh Masculino - Preto E Verme...,tenis ascension posh masculino preto vermelh
1,Tenis Para Caminhada Super Levinho Spider Corr...,tenis caminhada super levinho spider corr
2,Tênis Feminino Le Parc Hocks Black/ice Origina...,tenis feminino parc hocks black ice original envi
3,Tênis Olympikus Esportivo Academia Nova Tendên...,tenis olympikus esportivo academia nova tenden...
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...,inteligente led bicicleta tauda luz usb bicicl...


Criaçao dos modelos de vetorização e clusterização

In [6]:
tempo_inicial = time()

vectorizer = TfidfVectorizer( 

)

matriz_vetorizada_esparsa = vectorizer.fit_transform(df_base['ITE_ITEM_TITLE_transf'])

new_row = pd.DataFrame( {'Descricao':'Criacao do Vetorizador', 'Tempo': (time() - tempo_inicial) } , index=[0])
avaliacao_performance = pd.concat([ avaliacao_performance , new_row] , ignore_index=True )

print(f"n_samples: {matriz_vetorizada_esparsa.shape[0]}, n_features: {matriz_vetorizada_esparsa.shape[1]}")

n_samples: 29999, n_features: 16245


In [7]:
# Testes do resultado da vetorizacao. Erros quando não vetorizou nenhuma palavra
aux_teste = vectorizer.inverse_transform(matriz_vetorizada_esparsa)
aux_somatorio = matriz_vetorizada_esparsa.sum(axis = 1)

count = 0
for idx, x in enumerate(aux_somatorio):
    if x == 0:
        count = count + 1
        print(f'ORIGINAL: {df_base.loc[idx][0]}  PREPARADO: {df_base.loc[idx][1]}   VETORIZADO:{aux_teste[idx]}')
print(f'Total de itens nulos: {count}')

#%%

Total de itens nulos: 0


In [8]:
from sklearn.cluster import KMeans
import numpy as np

tempo_inicial = time()

clusterizador = KMeans(
    n_clusters=300,
    max_iter=100,
    n_init=5,
).fit(matriz_vetorizada_esparsa)


new_row = pd.DataFrame( {'Descricao':'Criação do Clusterizador', 'Tempo': (time() - tempo_inicial) } , index=[0])
avaliacao_performance = pd.concat([ avaliacao_performance , new_row] , ignore_index=True )

cluster_ids, cluster_sizes = np.unique(clusterizador.labels_, return_counts=True)
print(f"Numero de elementos de cada cluster: {cluster_sizes}")


Numero de elementos de cada cluster: [  36   95  335  105   86  165  101  136   27   51   47   21  372   28
   74   96  284  111  109   72   74  128   64  139   59   52  113   70
   54   64  114   99   90  131   26   31   15   55   27   23   93   76
  353   50   94   74  128  134   97   17   23   68   69   11  180   48
   26   13  104   79  347  105   96   64  229  149  120  166  141   46
   82  126  223  151   26  107   12   29   60  101    2   65   47   82
  193   57   48   65  487  162   68   31  206  212  158   53  141  162
  177   54  116   65  104  247   70   82  120   29  111  152  122  158
   56   58  222   58   59   94   13  175   38   65   15  146   49   20
   99   26   99   82  289  195  192  150  202   38   44   74   89  149
   23   26  133   29  173   92  255   66   34   95   99   75   24   57
 1481   73   60   59   24  164   59   43  113   33   17   31   68   60
  218   50   43   25   48   78   92   60   35   96  247  156   64   68
  104  142   42  124   45   34  194   18

#### Criaçao do dicionario key: cluster  value: list com os indices das linhas que foram classificadas neste clus 

In [9]:
# =============================================================================

# =============================================================================

from collections import defaultdict

dicti_clusters = defaultdict(list)

an_index = 0
for i in clusterizador.labels_:
    dicti_clusters[i].append(an_index)
    an_index = an_index+1

### Leitura e preparação dos dados da solução

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

df_base_test = pd.read_csv("items_titles_test.csv")

#Definir sample para testes
df_base_test = df_base_test.sample(n=100)

df_base_test = preparar_dados(df_base_test , 'ITE_ITEM_TITLE')

matriz_vetorizada_esparsa_test = vectorizer.transform(df_base_test['ITE_ITEM_TITLE_transf'])

df_base_test.head()

,ITE_ITEM_TITLE,ITE_ITEM_TITLE_transf
467,Tênis Fila Classic Runner Marinho E Vermelho,tenis fila classic runner marinho vermelh
8268,Tênis Destroyer Goyazes Ferradura Bronze,tenis destroyer goyazes ferradura bronz
4417,Tênis Calce Fácil Masculino Preto Tenis Slip O...,tenis calce facil masculino preto tenis slip b...
3300,Tênis Jogger Euroflex Masculino Couro Cadarço ...,tenis jogger euroflex masculino couro cadarco ...
2303,Tela E Barras De Led Tv Semp 40l2400 Não Envio...,tela barras led televisao semp 40l2400 nao env...


In [13]:
# Testes do resultado da vetorizacao. Erros quando não vetorizou nenhuma palavra
aux_teste = vectorizer.inverse_transform(matriz_vetorizada_esparsa_test)
aux_somatorio = matriz_vetorizada_esparsa_test.sum(axis = 1)

count = 0
for idx, x in enumerate(aux_somatorio):
    if x == 0:
        count = count + 1
        print(f'ORIGINAL: {df_base_test.loc[idx][0]}  PREPARADO: {df_base_test.loc[idx][1]}   VETORIZADO:{aux_teste[idx]}')
print(f'Total de itens nulos na vetorizaçao: {count}')

Total de itens nulos na vetorizaçao: 0


### Obtenção dos itens mais similares do catalogo para cada novo item

In [14]:
final_result = pd.DataFrame()
numero_resultados = 3

avaliacao_performance_predicao = pd.DataFrame(columns=['Descricao' , 'Tempo'] , index=[0])

tempo_inicial = time()

for an_index , an_item in enumerate(matriz_vetorizada_esparsa_test):
    tempo_inicial_iteracao = time()
    
    item_procurado_str = df_base_test.iloc[an_index]['ITE_ITEM_TITLE']
    # item = matriz_vetorizada_esparsa_test[100]
    
    cluster_item_procurado = clusterizador.predict(an_item)[0]
    
    indices_cluster_procurado = np.array(dicti_clusters[cluster_item_procurado])
    
    itens_no_cluster_procurado = df_base.iloc[indices_cluster_procurado]
    
    #preciso pegar os itens do cluster na base original
    matriz_cluster_catalogo = matriz_vetorizada_esparsa[indices_cluster_procurado,:]
    
    matriz_similaridade = cosine_similarity(matriz_cluster_catalogo , an_item )
    
    result = pd.DataFrame()
   
    result['ITE_ITEM_TITLE_CATALOGO'] = itens_no_cluster_procurado['ITE_ITEM_TITLE']
    result.reset_index(inplace=True)
    aux = pd.DataFrame(matriz_similaridade , columns=['SIMILARIDADE'])
    result['SIMILARIDADE'] = aux['SIMILARIDADE']
    
    aux = result.sort_values(by=['SIMILARIDADE'] , ascending = False)
    
    aux1=aux.iloc[0:numero_resultados]
    aux1 = aux1.reset_index()
    aux_series =  pd.Series([item_procurado_str for x in range(numero_resultados)])
    aux1['ITE_ITEM_TITLE_PROCURADO'] = aux_series
    
    final_result = pd.concat( [final_result , aux1] , ignore_index=True)
    
    new_row = pd.DataFrame( {'Descricao': f'Execução da predição {an_index}', 'Tempo': (time() - tempo_inicial_iteracao) } , index=[0])
    avaliacao_performance_predicao = pd.concat([ avaliacao_performance_predicao , new_row] , ignore_index=True )

new_row = pd.DataFrame( {'Descricao':'Execução Total da predição', 'Tempo': (time() - tempo_inicial) } , index=[0])
avaliacao_performance = pd.concat([ avaliacao_performance , new_row] , ignore_index=True )

#reordenando as colunas ...
final_result = final_result[['ITE_ITEM_TITLE_PROCURADO' , 'ITE_ITEM_TITLE_CATALOGO' , 'SIMILARIDADE']] 
final_result = final_result.reset_index(drop=True)


final_result.to_excel( "resultado.xlsx",sheet_name='Similaridade') 

print('Informações sobre o tempo de execução das predições')
print(avaliacao_performance_predicao['Tempo'].describe())


Informações sobre o tempo de execução das predições
count    100.000000
mean       0.057558
std        0.008175
min        0.040602
25%        0.050193
50%        0.057343
75%        0.063167
max        0.080218
Name: Tempo, dtype: float64
